# Homework 3
In this assignment, we will start with hyper-parameter tuning carried on from last homework and then building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## For Question 1:

### About the Data Set
Seven different types of dry beans were used in a study in Selcuk University, Turkey, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the classification model, images of 13611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features - 12 dimensions and 4 shape forms - were obtained from the grains.

Number of Instances (records in the data set): __13611__

Number of Attributes (fields within each record, including the class): __17__

### Data Set Attribute Information:

1. __Area (A)__ : The area of a bean zone and the number of pixels within its boundaries.
2. __Perimeter (P)__ : Bean circumference is defined as the length of its border.
3. __Major axis length (L)__ : The distance between the ends of the longest line that can be drawn from a bean.
4. __Minor axis length (l)__ : The longest line that can be drawn from the bean while standing perpendicular to the main axis.
5. __Aspect ratio (K)__ : Defines the relationship between L and l.
6. __Eccentricity (Ec)__ : Eccentricity of the ellipse having the same moments as the region.
7. __Convex area (C)__ : Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
8. __Equivalent diameter (Ed)__ : The diameter of a circle having the same area as a bean seed area.
9. __Extent (Ex)__ : The ratio of the pixels in the bounding box to the bean area.
10. __Solidity (S)__ : Also known as convexity. The ratio of the pixels in the convex shell to those found in beans.
11. __Roundness (R)__ : Calculated with the following formula: (4piA)/(P^2)
12. __Compactness (CO)__ : Measures the roundness of an object: Ed/L
13. __ShapeFactor1 (SF1)__
14. __ShapeFactor2 (SF2)__
15. __ShapeFactor3 (SF3)__
16. __ShapeFactor4 (SF4)__

17. __Classes : *Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, Sira*__

## For Questions 2-4:
### Background 
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

### Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

#### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [1]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install imbalanced-learn delayed

  Using cached delayed-0.11.0b1-py2.py3-none-any.whl (19 kB)
  Using cached redis-4.5.5-py3-none-any.whl (240 kB)


# Exercises

## Exercise 1 : Hyperparameter Tuning (20 points)

Use either grid search or random search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the number of epochs, such that the accuracy of the model is maximum for the given Dry_Beans_Dataset.

__Requirements :__
- The set of optimal hyperparameters
- The maximum accuracy achieved using this set of optimal hyperparameters

__Note :__ Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code.

In [3]:
import pandas as pd 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD

# read in dataset
df = pd.read_csv('Dry_Beans_Dataset.csv')

# split data into target and label
X = df.drop('Class', axis = 1)
y = df['Class']

# scale data 
scaler = StandardScaler()
X = scaler.fit_transform(X)

# split data into train and test sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_size = int(0.8 * len(X))
train_data, train_labels = X[:train_size], y[:train_size]
test_data, test_labels = X[train_size:], y[train_size:]

# intialize grid parameters
param_grid = {
    'learning_rate': [0.001, 0.01],
    'dropout_rate': [0.2, 0.3],
    'num_layers': [2, 3],
    'neurons_layer1': [64, 128],
    'neurons_layer2': [32, 64]
}

def create_model(num_layers=2, learning_rate=0.001, dropout_rate=0.2, neurons_layer1=64, neurons_layer2=32):
    model = Sequential()
    model.add(Dense(neurons_layer1, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))
    for i in range(num_layers - 1):
        model.add(Dense(neurons_layer2, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = SGD()
    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
    return model

# Create the grid search object
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)
# 3 is the number of folds in k-fold cross validation. one fold for validation and k-1 folds used for training
# for each combination of parameters, the model is trained on the training data and tested o the validation data.
# the average performance of the model (with the selected combination of parameters) over k-folds is used as the estimate for the model's perfromance for tha particular combination of hyperparameters. 

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# train the model with grid search
grid_result = grid.fit(train_data, train_labels)

# print the best hyperparameters and accuracy
print("Best parameters: ", grid_result.best_params_)
print("Best accuracy: {:.2f}%".format(grid_result.best_score_ * 100))

C:\Users\russe\AppData\Local\Temp\ipykernel_40828\2546947687.py:53: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)


Best parameters:  {'dropout_rate': 0.2, 'learning_rate': 0.001, 'neurons_layer1': 128, 'neurons_layer2': 64, 'num_layers': 2}
Best accuracy: 11.24%


## Exercise 2 - General Data Preprocessing (10 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [4]:
# If put the data(.csv) under the same folder, you could use
# df = pd.read_csv('./garments_worker_productivity.csv')
# read in data
df = pd.read_csv('garments_worker_productivity.csv')

# drop 'date' column
df = df.drop('date', axis=1)

# print categorical attributes
categorical_attributes = ['quarter', 'department', 'day']
for attribute in categorical_attributes:
    unique_elements = df[attribute].unique()
    print(f"\nUnique elements of '{attribute}':")
    for element in unique_elements:
        print(element)

# remap duplicate items (typos)
remap_dict = {
    'department': {'finishing ': 'finishing'}
}
df.replace(remap_dict, inplace=True)

# create 'satisifed' column 
df['satisfied'] = df['actual_productivity'] >= df['targeted_productivity']
df['satisfied'] = df['satisfied'].astype(int)

# drop 'actual_productivity' column and 'targeted_productivity' column
df = df.drop(['actual_productivity', 'targeted_productivity'], axis=1)

# find columns with NA values
columns_with_empty_values = df.columns[df.isna().any()].tolist()
print("\nColumns with empty values:", columns_with_empty_values)

# fill empty values with 0
df.fillna(0, inplace=True)


Unique elements of 'quarter':
Quarter1
Quarter2
Quarter3
Quarter4
Quarter5

Unique elements of 'department':
sweing
finishing 
finishing

Unique elements of 'day':
Thursday
Saturday
Sunday
Monday
Tuesday
Wednesday

Columns with empty values: ['wip']


## Exercise 3 - Naïve Bayes Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (5 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [5]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

df_nb = df.copy()
# get all categorical attributes
df_nb = pd.DataFrame(preprocessing.OrdinalEncoder().fit_transform(df_nb), columns=df_nb.columns)
# split data into 80:20 ratio
nb_train, nb_test = train_test_split(df_nb, test_size=0.2)

# encode the data 
X_nb_train, y_nb_train = nb_train.drop(columns=['satisfied']), nb_train['satisfied']
X_nb_test, y_nb_test = nb_test.drop(columns=['satisfied']), nb_test['satisfied']
print(X_nb_train.shape, X_nb_test.shape)

(957, 12) (240, 12)


### Exercise 3.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [6]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report

clf_cat = CategoricalNB()

clf_cat.fit(X_nb_train[categorical_attributes], y_nb_train)

print(classification_report(y_nb_test, clf_cat.predict(X_nb_test[categorical_attributes])))

              precision    recall  f1-score   support

         0.0       0.36      0.06      0.11        62
         1.0       0.75      0.96      0.84       178

    accuracy                           0.73       240
   macro avg       0.56      0.51      0.47       240
weighted avg       0.65      0.73      0.65       240



### Exercise 3.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [7]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
import numpy as np

# get numerical attributes
nums = [col for col in X_nb_train.columns if col not in categorical_attributes]

# scale the data
scaler = preprocessing.StandardScaler()
scaler.fit(X_nb_train[nums])
Z_nb_train = scaler.transform(X_nb_train[nums])
Z_nb_test = scaler.transform(X_nb_test[nums])

clf_num = GaussianNB()

clf_num.fit(Z_nb_train, np.asarray(y_nb_train))

print(classification_report(y_nb_test, clf_num.predict(Z_nb_test)))

              precision    recall  f1-score   support

         0.0       1.00      0.03      0.06        62
         1.0       0.75      1.00      0.86       178

    accuracy                           0.75       240
   macro avg       0.87      0.52      0.46       240
weighted avg       0.81      0.75      0.65       240



## Exercies 4 - SVM Classifier (35 points in total)

### Exercise 4.1 - Additional Data Preprocessing (5 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [8]:
# Remember to continue the task with your processed data from Exercise 1
nums = [col for col in df.drop(columns=['satisfied']).columns if col not in categorical_attributes]
df_svm = df.copy()
# encode with one-hot encoding 
df_svm = pd.get_dummies(df_svm, columns=categorical_attributes)

# split data into 80:20 ratio
svm_train, svm_test = train_test_split(df_svm, test_size=0.2)

X_svm_train, y_svm_train = svm_train.drop(columns=['satisfied']), svm_train['satisfied']
X_svm_test, y_svm_test = svm_test.drop(columns=['satisfied']), svm_test['satisfied']

### Exercise 4.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [9]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC

# scale the data
scaler = preprocessing.StandardScaler()
scaler.fit(X_svm_train)
Z_svm_train = scaler.transform(X_svm_train)
Z_svm_test = scaler.transform(X_svm_test)

# build SVM with linear kernel
svc_li = SVC(kernel='linear')
svc_li.fit(Z_svm_train, np.asarray(y_svm_train))

# print classification report for SVM with linear kernel
print('Linear Kernel')
print(classification_report(y_svm_test, svc_li.predict(Z_svm_test)))

# build SVM with RBF kernel
svc_rbf = SVC(kernel='rbf')
svc_rbf.fit(Z_svm_train, np.asarray(y_svm_train))

# print classification report for SVM with RBF kernel
print("RBF Kernel")
print(classification_report(y_svm_test, svc_rbf.predict(Z_svm_test)))

Linear Kernel
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        64
           1       0.74      1.00      0.85       176

    accuracy                           0.74       240
   macro avg       0.87      0.52      0.46       240
weighted avg       0.81      0.74      0.64       240

RBF Kernel
              precision    recall  f1-score   support

           0       0.53      0.14      0.22        64
           1       0.75      0.95      0.84       176

    accuracy                           0.74       240
   macro avg       0.64      0.55      0.53       240
weighted avg       0.69      0.74      0.68       240



### Exercise 4.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class. 
 - Oversample the **training data**. 
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [15]:
# Remember to do this task with your processed data from Exercise 3.1
from imblearn.over_sampling import RandomOverSampler

# print frequency of each class
print(y_svm_train.value_counts())

# oversample training data
ros = RandomOverSampler()
X_os, y_os = ros.fit_resample(X_svm_train, y_svm_train)

# print frequency of each class in oversampled data 
print(y_os.value_counts())

# scale the data
X_os = scaler.transform(X_os)

# fit the SVM models
svc_li.fit(X_os, np.asarray(y_os))
svc_rbf.fit(X_os, np.asarray(y_os))

# generate classification reports
print(classification_report(y_svm_test, svc_li.predict(Z_svm_test)))
print(classification_report(y_svm_test, svc_rbf.predict(Z_svm_test)))

1    699
0    258
Name: satisfied, dtype: int64
1    699
0    699
Name: satisfied, dtype: int64
              precision    recall  f1-score   support

           0       0.45      0.72      0.55        64
           1       0.87      0.68      0.76       176

    accuracy                           0.69       240
   macro avg       0.66      0.70      0.66       240
weighted avg       0.76      0.69      0.70       240

              precision    recall  f1-score   support

           0       0.47      0.77      0.58        64
           1       0.89      0.68      0.77       176

    accuracy                           0.70       240
   macro avg       0.68      0.72      0.68       240
weighted avg       0.78      0.70      0.72       240

